# Image Augmentation

In [ ]:
# Augmentation
import os
import random
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- Configuration ---
# Set the path to your dataset folder in Google Drive
base_dataset_path = '/content/drive/MyDrive/Thesis/dataset' # <-- IMPORTANT: Check this path is correct

# Define the folders for each class
class_folders = {
    'acnes': 'acnes',
    'hyperpigmentation': 'hyperpigmentation',
    'Nail_psoriasis': 'Nail_psoriasis',
    'SJS-TEN': 'SJS-TEN',
    'Vitiligo': 'Vitiligo' # <-- Double-check this name matches your folder exactly
}

# The target number of images per class
target_image_count = 2000

# --- Augmentation Setup ---
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# --- Main Loop ---
for class_name, folder_name in class_folders.items():
    folder_path = os.path.join(base_dataset_path, folder_name)

    # --- ⭐️ ERROR FIX: Check if the folder exists ---
    if not os.path.isdir(folder_path):
        print(f"🚨 ERROR: Directory not found for class '{class_name}': {folder_path}. Skipping.")
        continue

    # Get the list of all images in the folder
    images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    num_images = len(images)

    print(f"\nProcessing '{class_name}': Found {num_images} images.")

    # --- Logic to Increase, Decrease, or Skip ---
    if num_images < target_image_count:
        # --- AUGMENTATION (INCREASE) LOGIC ---
        num_to_generate = target_image_count - num_images
        print(f"Augmenting '{class_name}': Generating {num_to_generate} new images...")

        # Select random images from the folder to augment
        images_to_augment = random.choices(images, k=num_to_generate)

        for image_name in images_to_augment:
            image_path = os.path.join(folder_path, image_name)
            try:
                img = Image.open(image_path).convert('RGB')
                img = img.resize((224, 224))
                x = np.array(img)
                x = x.reshape((1,) + x.shape)

                # Generate one augmented image
                for batch in datagen.flow(x, batch_size=1):
                    augmented_image = Image.fromarray(batch[0].astype('uint8'))
                    new_image_name = f"aug_{random.randint(10000, 99999)}_{image_name}"
                    augmented_image.save(os.path.join(folder_path, new_image_name))
                    break # Stop after one is generated
            except Exception as e:
                print(f"Could not process image {image_name}. Error: {e}")

    elif num_images > target_image_count:
        # --- ⭐️ NEW: DECREASE LOGIC ---
        num_to_delete = num_images - target_image_count
        print(f"Decreasing '{class_name}': Randomly deleting {num_to_delete} images...")

        # Randomly select images to delete
        images_to_delete = random.sample(images, num_to_delete)

        for image_name in images_to_delete:
            file_to_delete_path = os.path.join(folder_path, image_name)
            try:
                os.remove(file_to_delete_path)
            except Exception as e:
                print(f"Could not delete image {image_name}. Error: {e}")

    else: # num_images == target_image_count
        print(f"'{class_name}' already has {target_image_count} images. No action needed.")


print("\nProcessing complete! ✅")



Processing 'acnes': Found 1148 images.
Augmenting 'acnes': Generating 852 new images...

Processing 'hyperpigmentation': Found 700 images.
Augmenting 'hyperpigmentation': Generating 1300 new images...

Processing 'Nail_psoriasis': Found 2520 images.
Decreasing 'Nail_psoriasis': Randomly deleting 520 images...

Processing 'SJS-TEN': Found 3164 images.
Decreasing 'SJS-TEN': Randomly deleting 1164 images...

Processing 'Vitiligo': Found 2016 images.
Decreasing 'Vitiligo': Randomly deleting 16 images...

Processing complete! ✅


# Split Dataset

In [ ]:
import os
import random
import shutil

# --- Configuration ---
# Path to your balanced dataset (where each class has 2000 images)
source_dataset_path = '/content/drive/MyDrive/Thesis/dataset' # <-- IMPORTANT: Path to your processed dataset

# Path where the new split dataset will be created
output_dataset_path = '/content/drive/MyDrive/Thesis/Split_Dataset'

# Define the split ratios
# 70% for training, 15% for validation, 15% for testing
split_ratios = (0.7, 0.15, 0.15)

# --- Main Splitting Logic ---
print("Starting dataset split...")

# Create the main output directories (train, validation, test)
if os.path.exists(output_dataset_path):
    shutil.rmtree(output_dataset_path) # Remove old directory to ensure a clean split
    print(f"Removed existing directory at: {output_dataset_path}")

train_dir = os.path.join(output_dataset_path, 'train')
validation_dir = os.path.join(output_dataset_path, 'validation')
test_dir = os.path.join(output_dataset_path, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get the list of class folders from the source directory
class_names = [d for d in os.listdir(source_dataset_path) if os.path.isdir(os.path.join(source_dataset_path, d))]

# Loop through each class folder
for class_name in class_names:
    print(f"\nProcessing class: {class_name}")

    # Create class-specific subdirectories in train, validation, and test folders
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    # Get all image file names for the current class
    source_class_dir = os.path.join(source_dataset_path, class_name)
    all_files = [f for f in os.listdir(source_class_dir) if os.path.isfile(os.path.join(source_class_dir, f))]

    # Shuffle the files randomly for an unbiased split
    random.shuffle(all_files)

    # Calculate split indices
    total_files = len(all_files)
    train_end = int(total_files * split_ratios[0])
    validation_end = train_end + int(total_files * split_ratios[1])

    # Slice the list of files into the three sets
    train_files = all_files[:train_end]
    validation_files = all_files[train_end:validation_end]
    test_files = all_files[validation_end:]

    # --- Function to copy files ---
    def copy_files(files, source_dir, dest_dir):
        for f in files:
            shutil.copy(os.path.join(source_dir, f), os.path.join(dest_dir, f))

    # Copy files to their new destination
    copy_files(train_files, source_class_dir, os.path.join(train_dir, class_name))
    copy_files(validation_files, source_class_dir, os.path.join(validation_dir, class_name))
    copy_files(test_files, source_class_dir, os.path.join(test_dir, class_name))

    print(f"  - Training set:   {len(train_files)} images")
    print(f"  - Validation set: {len(validation_files)} images")
    print(f"  - Testing set:    {len(test_files)} images")

print("\nDataset splitting complete! ✅")

Starting dataset split...

Processing class: hyperpigmentation
  - Training set:   1400 images
  - Validation set: 300 images
  - Testing set:    300 images

Processing class: Nail_psoriasis
  - Training set:   1400 images
  - Validation set: 300 images
  - Testing set:    300 images

Processing class: SJS-TEN
  - Training set:   1400 images
  - Validation set: 300 images
  - Testing set:    300 images

Processing class: Vitiligo
  - Training set:   1400 images
  - Validation set: 300 images
  - Testing set:    300 images

Processing class: acnes
  - Training set:   1400 images
  - Validation set: 300 images
  - Testing set:    300 images

Dataset splitting complete! ✅


# Model Training


## DenseNet 121

In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import DenseNet121

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import os

# --- 1. SETUP FOR SPEED: MIXED PRECISION ---

tf.keras.mixed_precision.set_global_policy('mixed_float16')

# --- 2. Configuration and Setup ---

# Define paths to your split dataset

base_dir = '/content/drive/MyDrive/Thesis/Split_Dataset'

train_dir = os.path.join(base_dir, 'train')

validation_dir = os.path.join(base_dir, 'validation')

test_dir = os.path.join(base_dir, 'test')

# Model parameters

IMG_HEIGHT = 224

IMG_WIDTH = 224

BATCH_SIZE = 64 # Larger batch size for faster training

INITIAL_EPOCHS = 10

FINE_TUNE_EPOCHS = 10

TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS

# --- 3. Data Loading and Preprocessing ---

# Create ImageDataGenerators with DenseNet-specific preprocessing

train_datagen = ImageDataGenerator(

    preprocessing_function=tf.keras.applications.densenet.preprocess_input,

    rotation_range=40,

    width_shift_range=0.2,

    height_shift_range=0.2,

    shear_range=0.2,

    zoom_range=0.2,

    horizontal_flip=True,

    fill_mode='nearest'

)

validation_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.densenet.preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.densenet.preprocess_input)

# Create data generators from directories

train_generator = train_datagen.flow_from_directory(

    train_dir, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, class_mode='sparse')

validation_generator = validation_datagen.flow_from_directory(

    validation_dir, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, class_mode='sparse')

test_generator = test_datagen.flow_from_directory(

    test_dir, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, class_mode='sparse', shuffle=False)

num_classes = train_generator.num_classes

print(f"Found {num_classes} classes.")

# --- 4. Model Building (Transfer Learning with DenseNet121) ---

# Load the DenseNet121 base model

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Freeze the base model

base_model.trainable = False

# Create our custom classifier head

x = base_model.output

x = GlobalAveragePooling2D()(x)

x = Dropout(0.5)(x)

# Set the final layer's dtype to float32 for numerical stability

predictions = Dense(num_classes, activation='softmax', dtype='float32')(x)

# Combine the base model and our custom head

model = Model(inputs=base_model.input, outputs=predictions)

# --- 5. Compile and Train (Feature Extraction) ---

# Wrap the optimizer in a LossScaleOptimizer for mixed precision

optimizer = tf.keras.mixed_precision.LossScaleOptimizer(Adam(learning_rate=0.001))

model.compile(optimizer=optimizer,

              loss='sparse_categorical_crossentropy',

              metrics=['accuracy'])

print("\n--- Starting Initial Training (Feature Extraction) ---")

history = model.fit(

    train_generator,

    epochs=INITIAL_EPOCHS,

    validation_data=validation_generator

)

# --- 6. Fine-Tuning Phase ---

print("\n--- Starting Fine-Tuning ---")

base_model.trainable = True

# DenseNet121 has 427 layers. Let's unfreeze from the last dense block onwards.

fine_tune_at = 313

for layer in base_model.layers[:fine_tune_at]:

    layer.trainable = False

# Re-compile with the LossScaleOptimizer for fine-tuning

optimizer_fine = tf.keras.mixed_precision.LossScaleOptimizer(Adam(learning_rate=1e-5))

model.compile(optimizer=optimizer_fine,

              loss='sparse_categorical_crossentropy',

              metrics=['accuracy'])

history_fine = model.fit(

    train_generator,

    epochs=TOTAL_EPOCHS,

    initial_epoch=history.epoch[-1],

    validation_data=validation_generator

)

# --- 7. Final Evaluation ---

print("\n--- Evaluating Model on Test Data ---")

test_loss, test_accuracy = model.evaluate(test_generator)

print(f"Test Accuracy: {test_accuracy*100:.2f}%")

print(f"Test Loss: {test_loss:.4f}")

# --- 8. Plotting History ---

acc = history.history['accuracy'] + history_fine.history['accuracy']

val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']

loss = history.history['loss'] + history_fine.history['loss']

val_loss = history.history['val_loss'] + history_fine.history['val_loss']

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)

plt.plot(acc, label='Training Accuracy')

plt.plot(val_acc, label='Validation Accuracy')

plt.axvline(INITIAL_EPOCHS - 1, color='gray', linestyle='--', label='Start Fine-Tuning')

plt.title('Training and Validation Accuracy')

plt.legend()

plt.grid(True)

plt.subplot(1, 2, 2)

plt.plot(loss, label='Training Loss')

plt.plot(val_loss, label='Validation Loss')

plt.axvline(INITIAL_EPOCHS - 1, color='gray', linestyle='--', label='Start Fine-Tuning')

plt.title('Training and Validation Loss')

plt.legend()

plt.grid(True)

plt.tight_layout()




Found 7000 images belonging to 5 classes.
Found 1500 images belonging to 5 classes.
Found 1500 images belonging to 5 classes.
Found 5 classes.
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

--- Starting Initial Training (Feature Extraction) ---


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
  4/110 ━━━━━━━━━━━━━━━━━━━━ 34:45 20s/step - accuracy: 0.1549 - loss: 2.7844

KeyboardInterrupt: 

**Confusion Matrix**

In [ ]:
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

# --- 1. Get Predictions and True Labels ---

# Reset the test generator to ensure the order is correct

test_generator.reset()

# Use the trained model to make predictions on the test set

y_pred_probs = model.predict(test_generator, steps=int(np.ceil(test_generator.samples / test_generator.batch_size)))

# Convert prediction probabilities to class indices

y_pred = np.argmax(y_pred_probs, axis=1)

# Get the ground truth labels from the generator

y_true = test_generator.classes

# Get the names of the classes for labeling

class_names = list(test_generator.class_indices.keys())


# --- 2. Generate Classification Report ---

print("\n## Classification Report ##\n")

# This report shows precision, recall, f1-score, and support for each class

report = classification_report(y_true, y_pred, target_names=class_names)

print(report)


# --- 3. Plot Confusion Matrix ---

print("\n## Confusion Matrix ##\n")

# Create the confusion matrix

cm = confusion_matrix(y_true, y_pred)

# Visualize the confusion matrix using a heatmap

plt.figure(figsize=(10, 8))

sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges',

            xticklabels=class_names, yticklabels=class_names)

plt.title('Confusion Matrix')

plt.ylabel('True Label')

plt.xlabel('Predicted Label')

plt.show()


NameError: name 'test_generator' is not defined